In [1]:
# Import required libraries
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("data/modelCreationData.csv")
df.shape

(7838, 24)

In [3]:
df

,id,artist_name,key,mode,track_name,popularity,acousticness,danceability,duration_ms,energy,...,tempo,valence,hit,year,explicit,sentimentPos,sentimentNeg,sentimentComp,sentimentNeu,pos_neg_diff
0,0wqOReZDnrefefEsrIGeR4,Lifehouse,1.0,1.0,Hanging By A Moment,68,0.00118,0.541,216067.0,0.864,...,124.557,0.435,1,2000,0.0,0.198,0.154,0.6597,0.648,0.044
1,0KQx6HOpJueiSkztcS0r7D,Alicia Keys,11.0,0.0,Fallin',70,0.26300,0.652,210200.0,0.609,...,95.986,0.482,1,2001,0.0,0.418,0.116,0.9456,0.467,0.302
2,5X8kkUaUlAyAUr9TYqDFTH,Janet Jackson,2.0,1.0,All For You,67,0.01740,0.753,329933.0,0.934,...,113.525,0.730,1,2001,0.0,0.229,0.133,0.8520,0.638,0.096
3,2hKdd3qO7cWr2Jo0Bcs0MA,Train,0.0,1.0,Drops of Jupiter (Tell Me),80,0.15300,0.481,259933.0,0.638,...,79.064,0.497,1,2001,0.0,0.210,0.000,0.9288,0.790,0.210
4,4onVfPBjiLokGWsGRdPH7v,Ja Rule,2.0,1.0,I'm Real - Murder Remix,49,0.40100,0.700,252933.0,0.631,...,83.403,0.558,1,2001,1.0,0.232,0.194,0.5832,0.574,0.038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7833,7f2vcDjWjVqkENE38q7vxJ,The Dreadnoughts,0.0,1.0,Randy Dandy-Oh,50,0.87300,0.810,119653.0,0.446,...,113.653,0.914,0,2010,1.0,0.123,0.000,0.4738,0.877,0.123
7834,6j0teRL3WdExzqmy334sh5,Lil Uzi Vert,1.0,0.0,Seven Million (feat. Future),62,0.05100,0.886,182491.0,0.629,...,158.002,0.487,0,2016,1.0,0.162,0.164,-0.5053,0.674,-0.002
7835,1ufGqwim7RHxVsdaxuLmAY,guccihighwaters,10.0,0.0,i'm so sick of this,64,0.89100,0.719,234736.0,0.387,...,135.064,0.170,0,2017,1.0,0.129,0.284,-0.9729,0.588,-0.155
7836,5nayhWICkQGMTkisxVMbRw,Playboi Carti,9.0,1.0,FlatBed Freestyle,70,0.09670,0.755,193973.0,0.552,...,153.157,0.402,0,2018,1.0,0.054,0.213,-0.9325,0.733,-0.159


In [4]:
X = df[["acousticness", "danceability", "duration_ms", "energy", "explicit", "instrumentalness", "key", "liveness", "loudness", "mode", "popularity", "speechiness", "tempo", "valence", "sentimentPos", "sentimentNeg", "sentimentComp", "sentimentNeu", "pos_neg_diff"]]
# X = df[["acousticness", "danceability", "duration_ms", "energy", "explicit", "instrumentalness", "key", "liveness", "loudness", "mode", "popularity", "speechiness", "tempo", "valence"]]
y = df["hit"]
X

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,sentimentPos,sentimentNeg,sentimentComp,sentimentNeu,pos_neg_diff
0,0.00118,0.541,216067.0,0.864,0.0,0.000000,1.0,0.0896,-4.915,1.0,68,0.0357,124.557,0.435,0.198,0.154,0.6597,0.648,0.044
1,0.26300,0.652,210200.0,0.609,0.0,0.001010,11.0,0.2330,-7.519,0.0,70,0.0370,95.986,0.482,0.418,0.116,0.9456,0.467,0.302
2,0.01740,0.753,329933.0,0.934,0.0,0.065000,2.0,0.1280,-3.011,1.0,67,0.0736,113.525,0.730,0.229,0.133,0.8520,0.638,0.096
3,0.15300,0.481,259933.0,0.638,0.0,0.000000,0.0,0.1540,-5.862,1.0,80,0.0276,79.064,0.497,0.210,0.000,0.9288,0.790,0.210
4,0.40100,0.700,252933.0,0.631,1.0,0.000000,2.0,0.1190,-6.085,1.0,49,0.1070,83.403,0.558,0.232,0.194,0.5832,0.574,0.038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7833,0.87300,0.810,119653.0,0.446,1.0,0.000000,0.0,0.4420,-6.084,1.0,50,0.0669,113.653,0.914,0.123,0.000,0.4738,0.877,0.123
7834,0.05100,0.886,182491.0,0.629,1.0,0.000008,1.0,0.1210,-4.404,0.0,62,0.3060,158.002,0.487,0.162,0.164,-0.5053,0.674,-0.002
7835,0.89100,0.719,234736.0,0.387,1.0,0.000000,10.0,0.1140,-9.380,0.0,64,0.0643,135.064,0.170,0.129,0.284,-0.9729,0.588,-0.155
7836,0.09670,0.755,193973.0,0.552,1.0,0.000000,9.0,0.1580,-5.604,1.0,70,0.2880,153.157,0.402,0.054,0.213,-0.9325,0.733,-0.159


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [6]:
# Use these features: https://maelfabien.github.io/machinelearning/Explorium6/#new-model
# Refer this for cross validation: https://github.com/manasreldin/Song-Popularity-Predictor/blob/master/PredictHotBB.ipynb

In [7]:
def print_metrics(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    auc_score = roc_auc_score(y_test, y_pred)
    print("ROC-AUC score: %.2f%%" % (auc_score * 100.0))
    print("Classification Report:\n ")
    print(classification_report(y_test, y_pred))

## Logistic Regression:

In [8]:
from sklearn.model_selection import GridSearchCV

grid = {"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg = LogisticRegression()
logreg_cv = GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(X_train,y_train)

print("Tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

Tuned hpyerparameters :(best parameters)  {'C': 100.0, 'penalty': 'l2'}
accuracy : 0.8079254018472806


In [9]:
log_reg_clf = LogisticRegression(random_state=0, C=100.0, penalty='l2').fit(X_train, y_train)
y_pred = log_reg_clf.predict(X_test)

# log_reg.predict_proba(X_test)[:,1]
# Print metrics
print_metrics(y_test, y_pred)

Accuracy: 79.08%
ROC-AUC score: 55.07%
Classification Report:
 
              precision    recall  f1-score   support

           0       0.81      0.97      0.88      1547
           1       0.51      0.14      0.21       413

    accuracy                           0.79      1960
   macro avg       0.66      0.55      0.55      1960
weighted avg       0.75      0.79      0.74      1960



In [10]:
# Save logistic regression model
pickle.dump(log_reg_clf, open('models/log_reg.pkl', 'wb'))

## Decision Tree:

In [11]:
dt_clf = DecisionTreeClassifier(random_state=0)
dt_clf.fit(X_train, y_train)
y_pred = dt_clf.predict(X_test)

# Print metrics
print_metrics(y_test, y_pred)

Accuracy: 79.03%
ROC-AUC score: 69.23%
Classification Report:
 
              precision    recall  f1-score   support

           0       0.87      0.86      0.87      1547
           1       0.50      0.52      0.51       413

    accuracy                           0.79      1960
   macro avg       0.69      0.69      0.69      1960
weighted avg       0.79      0.79      0.79      1960



In [12]:
# Save decision tree model
pickle.dump(dt_clf, open('models/dt.pkl', 'wb'))

## Random Forest:

In [13]:
rf_clf = RandomForestClassifier(random_state=0)
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)

# Print metrics
print_metrics(y_test, y_pred)

Accuracy: 86.12%
ROC-AUC score: 72.75%
Classification Report:
 
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1547
           1       0.76      0.50      0.60       413

    accuracy                           0.86      1960
   macro avg       0.82      0.73      0.76      1960
weighted avg       0.85      0.86      0.85      1960



In [14]:
# Save random forest model
pickle.dump(rf_clf, open('models/rf.pkl', 'wb'))

## Support Vector Classification(SVC):

In [15]:
svc_clf = make_pipeline(StandardScaler(), SVC(gamma='auto', probability=True))
svc_clf.fit(X_train, y_train)

y_pred = svc_clf.predict(X_test)

# Print metrics
print_metrics(y_test, y_pred)

Accuracy: 85.66%
ROC-AUC score: 71.39%
Classification Report:
 
              precision    recall  f1-score   support

           0       0.87      0.96      0.91      1547
           1       0.76      0.47      0.58       413

    accuracy                           0.86      1960
   macro avg       0.82      0.71      0.75      1960
weighted avg       0.85      0.86      0.84      1960



In [16]:
# Save SVC model
pickle.dump(svc_clf, open('models/svc.pkl', 'wb'))

## XGBoost:

In [17]:
xgb_cls = XGBClassifier()
xgb_cls.fit(X_train, y_train)

y_pred = xgb_cls.predict(X_test)
# predictions = [round(value) for value in y_pred]

# Print metrics
print_metrics(y_test, y_pred)

[17:15:19] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 84.80%
ROC-AUC score: 73.33%
Classification Report:
 
              precision    recall  f1-score   support

           0       0.88      0.93      0.91      1547
           1       0.68      0.54      0.60       413

    accuracy                           0.85      1960
   macro avg       0.78      0.73      0.75      1960
weighted avg       0.84      0.85      0.84      1960



In [18]:
# Save xgboost model
pickle.dump(xgb_cls, open('models/xgb.pkl', 'wb'))